In [2]:
import tensorflow as tf 
from tensorflow.keras.datasets import mnist 


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

2024-06-18 14:56:39.955002: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 14:56:39.992534: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 14:56:40.543341: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model = tf.keras.Sequential(
    [    tf.keras.layers.Conv2D(32, (3, 3), 
        activation='relu', input_shape=(28, 28, 1)),    
        tf.keras.layers.MaxPooling2D((2, 2)),    
        tf.keras.layers.Flatten(),    
        tf.keras.layers.Dense(128, activation='relu'),    
        tf.keras.layers.Dense(10, activation='softmax')
    ])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-18 14:56:41.858870: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 14:56:41.877801: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 14:56:41.877826: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kerne

In [4]:
model.compile(optimizer='adam',              
              loss='sparse_categorical_crossentropy',              
              metrics=['accuracy'])

In [5]:
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5


I0000 00:00:1718722606.688346    8668 service.cc:145] XLA service 0x7f1cdc006070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718722606.688386    8668 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-18 14:56:46.704289: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-18 14:56:47.918095: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


  75/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6124 - loss: 1.3110

I0000 00:00:1718722609.918446    8668 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9092 - loss: 0.3128 - val_accuracy: 0.9802 - val_loss: 0.0601
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9828 - loss: 0.0555 - val_accuracy: 0.9842 - val_loss: 0.0486
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9908 - loss: 0.0333 - val_accuracy: 0.9845 - val_loss: 0.0470
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9937 - loss: 0.0211 - val_accuracy: 0.9839 - val_loss: 0.0523
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9958 - loss: 0.0137 - val_accuracy: 0.9869 - val_loss: 0.0417


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert() 
with open('model.tflite', 'wb') as f:    
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpdrko70pa/assets


INFO:tensorflow:Assets written to: /tmp/tmpdrko70pa/assets


: 

In [ ]:
%edge-impulse-uploader --api-key <your-api-key> model.tflite